<a href="https://colab.research.google.com/github/BhaveshNikam09/Deep_learning_practise/blob/main/Inception_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, AveragePooling2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar100

In [13]:
(X_train,y_train),(X_test,y_test)=cifar100.load_data()

In [14]:
X_train.shape

(50000, 32, 32, 3)

In [20]:
def inception_block(x,filters):
  f1,f3_reduce,f3,f5_reduce,f5,pool_proj = filters
  #branch 1
  branch1=Conv2D(f1,(1,1),padding='same',activation='relu')(x)

  #branch 2
  branch2=Conv2D(f3_reduce,(1,1),padding='same',activation='relu')(x)
  branch2=Conv2D(f3,(3,3),padding='same',activation='relu')(branch2)

  #branch3
  branch3=Conv2D(f5_reduce,(1,1),padding='same',activation='relu')(x)
  branch3=Conv2D(f5,(5,5),padding='same',activation='relu')(branch3)

  #branch4
  branch4=MaxPooling2D((3,3),strides=(1,1),padding='same')(x)
  branch4=Conv2D(pool_proj,(1,1),padding='same',activation='relu')(branch4)

  return Concatenate()([branch1,branch2,branch3,branch4])

In [21]:
def auxillary_classifier(x,num_classes):
  x=AveragePooling2D((5,5),strides=(3,3))(x)
  x=Conv2D(128,(1,1),padding='same',activation='relu')(x)
  x=Flatten()(x)
  x=Dense(1024,activation='relu')(x)
  x=Dropout(0.7)(x)
  x=Dense(num_classes,activation='softmax')(x)
  return x

In [24]:
def build_inception(input_shape=(32, 32, 3), num_classes=100):
    inputs = Input(shape=input_shape)


    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)


    x = inception_block(x, (64, 96, 128, 16, 32, 32))
    x = inception_block(x, (128, 128, 192, 32, 96, 64))
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_block(x, (192, 96, 208, 16, 48, 64))
    aux1 = auxillary_classifier(x, num_classes)

    x = inception_block(x, (160, 112, 224, 24, 64, 64))
    x = inception_block(x, (128, 128, 256, 24, 64, 64))
    x = inception_block(x, (112, 144, 288, 32, 64, 64))
    aux2 = auxillary_classifier(x, num_classes)

    x = inception_block(x, (256, 160, 320, 32, 128, 128))
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_block(x, (256, 160, 320, 32, 128, 128))
    x = inception_block(x, (384, 192, 384, 48, 128, 128))


    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)


    model = Model(inputs, [outputs, aux1, aux2])

    return model


In [25]:
model = build_inception(input_shape=(224, 224, 3), num_classes=10)

model.compile(optimizer="adam",
              loss=["sparse_categorical_crossentropy", "sparse_categorical_crossentropy", "sparse_categorical_crossentropy"],
              loss_weights=[1, 0.3, 0.3],
              metrics=["accuracy"])


model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 112, 112, 64)   │          9,472 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 56, 56, 64)     │              0 │ conv2d_6[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 56, 56, 64)     │          4,160 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 56, 56, 192)    │        110,784 │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 28, 28, 192)    │              0 │ conv2d_8[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_10 (Conv2D)        │ (None, 28, 28, 96)     │         18,528 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_12 (Conv2D)        │ (None, 28, 28, 16)     │          3,088 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_6           │ (None, 28, 28, 192)    │              0 │ max_pooling2d_5[0][0]  │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 28, 28, 64)     │         12,352 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_11 (Conv2D)        │ (None, 28, 28, 128)    │        110,720 │ conv2d_10[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_13 (Conv2D)        │ (None, 28, 28, 32)     │         12,832 │ conv2d_12[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_14 (Conv2D)        │ (None, 28, 28, 32)     │          6,176 │ max_pooling2d_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 28, 28, 256)    │              0 │ conv2d_9[0][0],        │
│                           │                        │                │ conv2d_11[0][0],       │
│                           │                        │                │ conv2d_13[0][0],       │
│                           │                        │                │ conv2d_14[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_16 (Conv2D)        │ (None, 28, 28, 128)    │         32,896 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_18 (Conv2D)        │ (None, 28, 28, 32)     │          8,224 │ concatenate[0][0]      │
├──────────────────────

 Total params: 10,334,030 (39.42 MB)

 Trainable params: 10,334,030 (39.42 MB)

 Non-trainable params: 0 (0.00 B)

# **Tranfer learning**

In [30]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.models import Sequential

base_model=inception_v3.InceptionV3(weights='imagenet',include_top=False)

for layer in base_model.layers:
  layer.trainable=False

model=Sequential()
model.add(base_model)
model.add(Dense(100,activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, None, None, 2048)    │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, None, None, 100)     │         204,900 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,007,684 (83.95 MB)

 Trainable params: 204,900 (800.39 KB)

 Non-trainable params: 21,802,784 (83.17 MB)